<br></br>
# **자연어와 Deep Learning**
## **LSTM 단어유추모델**

<br></br>
## **1 데이터 정의**

In [12]:
import tensorflow as tf
import numpy as np
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
num_dic  = {n: i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)
num_dic

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

In [2]:
def make_batch(seq_data):
    input_batch, target_batch = [], []
    for seq in seq_data: 
        input_num = [num_dic[n] for n in seq[:-1]]  
        target    = num_dic[seq[-1]]                
        input_batch.append(np.eye(dic_len)[input_num])
        target_batch.append(target)          
    return input_batch, target_batch

<br></br>
## **2 모델의 정의**

In [3]:
tf.reset_default_graph()
learning_rate = 0.01
n_step = 3           
n_hidden, total_epoch = 64, 30
n_input = n_class = dic_len

In [4]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32,   [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class])) 
b = tf.Variable(tf.random_normal([n_class]))

In [5]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)    
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)         
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

In [6]:
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model   = tf.matmul(outputs, W) + b

In [7]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

<br></br>
## **3 모델의 학습**

In [8]:
%%time
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})
    if epoch % 4 == 0:
        print('Epoch: {:.4f} cost = {:.6f}'.format(epoch + 1, loss))
print('최적화 완료!')

Epoch: 1.0000 cost = 2.627321
Epoch: 5.0000 cost = 1.243245
Epoch: 9.0000 cost = 0.565202
Epoch: 13.0000 cost = 0.308347
Epoch: 17.0000 cost = 0.298289
Epoch: 21.0000 cost = 0.372310
Epoch: 25.0000 cost = 0.201546
Epoch: 29.0000 cost = 0.057700
최적화 완료!
CPU times: user 1.99 s, sys: 736 ms, total: 2.73 s
Wall time: 3.96 s


<br></br>
## **4 학습 모델의 평가**

In [9]:
%%time
prediction       = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)               
accuracy         = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val     = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

CPU times: user 79.9 ms, sys: 8.18 ms, total: 88.1 ms
Wall time: 85.4 ms


In [15]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:1] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:-2] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)
sess.close()


=== 예측 결과 ===
입력값: ['wo ', 'wo ', 'de ', 'di ', 'co ', 'co ', 'lo ', 'lo ', 'ki ', 'ki ']
예측값: ['wd', 'wd', 'dp', 'de', 'cd', 'cl', 'ld', 'le', 'ks', 'kd']
정확도: 1.0
